In [1]:
import io, json, codecs, os, fix

In [4]:
def redif2jsonld(jsonld_src,journal_name,issn):
    global articles; articles=[]
    global article; article={}
    global authors; authors=[]
    global author; author=''
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(author)>0: authors.append(author); author=''
                article['e:author']=authors
                authors=[]
                if fix.validate(article)==True: articles.append(article)
                else: e=e+1
            article={}
            article['e:author']=[]
            article['e:keyword']=[]
            article['e:journal']=journal_name
            article['e:year']=''
            article['e:title']=''
        elif key=='author-name':
            if len(author)>0: authors.append(author)
            author=''
            if ',' in j: j=' '.join(reversed([part.strip() for part in j.split(',')]))
            author=j
        elif key=='title':article['e:title']=j
        elif key=='year':article['e:year']=j
        elif key=='pages':article['e:pages']=j
        elif key=='volume':article['e:volume']=j
        elif key=='issue':article['e:issue']=j
        elif key=='file-url':article['e:url']=j
        elif key=='doi':article['e:doi']=j
        elif key=='abstract':article['e:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='': article['e:keyword'].append(fix.fix_keyword(keyword))

    if len(article)>0:
        if len(author)>0: authors.append(author); author=''
        article['e:author']=authors; authors=[]
        articles.append(article)   
        
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    jsonld_file=jsonld_src+issn+'.jsonld'
    if not os.path.exists(jsonld_file):
        with io.open(jsonld_file, 'w', encoding="utf-8") as f:
            f.write(jd)
    return (jsonld_file)

In [10]:
redif_folder='redif/'
jsonld_src='jsonld_src/'

    
with open('../OpenScience/subject_journals.json') as data_file:    
    subject_journals = json.load(data_file)

journals=[]
for subject in subject_journals:
    journals=journals+(subject['journals'])
print len(journals)
    
for journal in journals:
    source=journal['jname']
    if source != 'jrnmis' : continue
    journal_name=journal['journal']
    data=journal['data']
    if 'issn' in journal: issn=journal['issn']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='redif' or file_type=='parse':
            if file_type=='parse': publisher='src'
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(redif_folder+publisher+'/'+source):
                if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                    redif_file=publisher+'/'+source+'/'+file_name
                    content = []
                    articles=[]
                    with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                        for line in file:
                            line = line.strip()
                            content.append(line)
                    jsonld_file=redif2jsonld(jsonld_src,journal_name,issn)
                i=i+1
            print ', INVALID=',e,'files=',i

40
src jrnmis Journal of Management Information Systems , INVALID= 1 files= 1


In [14]:
jsonld_src='jsonld_src/'
jsonld_folder='jsonld/'
with open(jsonld_src+'0742-1222.jsonld') as data_file:    
    data = json.load(data_file)


In [15]:
data[0]

{u'e:abstract': u'The Journal of Management Information Systems will provide an integrated view of the entire discipline to both researchers and practitioners and will help communicate research and practical advances.In order to define the focal points of the new journal, the developmental trends in mis as a field of practice and as an academic discipline are analyzed. Support of less structured decisions under the condition of organizational dynamics is found to be of particular importance. At the same time, the environment of information delivery will be determined by cooperative computing, with the end users gaining in a large measure direct access to information.The role of mis as a crucial component in the evolving knowledge society is stressed. In particular, flexibility of information systems is a factor of social productivity at large.',
 u'e:author': [u'Vladimir Zwass'],
 u'e:doi': u'10.1080/07421222.1984.11517693',
 u'e:issue': u'1',
 u'e:journal': u'Journal of Management Inf